## Dashboard

### Install all dependencies

In [1]:
!pip install pandas
!pip install plotly
!pip install dash
!pip install dash_bootstrap_components

In [1]:
import pandas as pd
pd.set_option('max_rows',20)
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

In [2]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import pandas


<ipython-input-2-91f9bae444a8>:3: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

<ipython-input-2-91f9bae444a8>:4: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [3]:
# load the csv file that we saved
Main_Data = pd.read_csv("Saved_Date")

In [4]:
# Create new fields
Main_Data["Year"]=pd.to_datetime(Main_Data["Date"].str[-4:])
Main_Data["Month"]=pd.to_datetime(Main_Data["Date"].str[3:])
Main_Data["Date_new"] = pd.to_datetime(Main_Data["Date"])
Main_Data["Sum"]=1

In [5]:
# Create a function that groups the data by the date and determine the number of customer that day
def group_by_date():
    df = pandas.DataFrame(Main_Data.groupby(["Date_new"]).sum()["Sum"])
    return df

In [6]:
# Create a function that groups the data by the Month and determine the number of customer that Month
def group_by_Month():
    df = pandas.DataFrame(Main_Data.groupby(["Month"]).sum()["Sum"])
    return df

In [7]:
# Create a function that groups the data by the Year and determine the number of customer that Year
def group_by_Year():
    df = pandas.DataFrame(Main_Data.groupby(["Year"]).sum()["Sum"])
    return df

In [8]:
# Create a function that groups the data by the Age and determine the number of customer 
def group_by_Age():
    df = pandas.DataFrame(Main_Data.groupby(["Age"]).sum()["Sum"])
    return df

In [9]:
# Create a function that groups the data by the Gender and determine the number of customer that day
def group_by_Gender():
    df = pandas.DataFrame(Main_Data.groupby(["Gender"]).sum()["Sum"])
    return df

In [10]:
def group_by_Age_Month():
    return Main_Data.groupby(["Month","Age"]).sum()["Sum"]

In [11]:
def group_by_Gender_Month():
    return Main_Data.groupby(["Month","Gender"]).sum()["Sum"]

### Generate Line Graph using Plotly

In [12]:
# Create a function that display a line figure  
def fig_Line(df,col,date):
    fig = px.line(df, y=col, x=df.index, title='{} Customer flow'.format(date),height=600,color_discrete_sequence =['maroon'])
    fig.update_layout(title_x=0.5,plot_bgcolor='#F2DFCE',paper_bgcolor='#F2DFCE',xaxis_title="Date",yaxis_title='Customer Flow')
    return fig

### Generate Pie Graph using Plotly

In [13]:
# Create a function that display a Pie figure  

def fig_pie(df,col):
    fig = px.pie(df, values=col,
         names=df.index,
         labels=['Female','Male'],
         title='Gender Repartition',
         color=df.index,color_discrete_map={'Female':'indianred',
                             'Male':'mediumseagreen'})
    fig.update_traces(textposition='inside', textinfo='percent+label',
                    textfont_size=10,showlegend=True,
                    insidetextorientation='horizontal')
    fig.update_layout(title_x=0.5)
    return fig

### Generate bar Graph using Plotly

In [14]:
# Create a function that display a Bar figure  

def fig_Bar(df,title):
    fig = px.bar(df,title='{}'.format(title))
    fig.update_layout(title_x=0.5)

    return fig

In [15]:
# Create a function that display a Composed_Bar figure  

def Composed_Bar(df):
    fig = px.bar(df, color="Age")
    return fig

In [16]:
# Create a function that display a fig_Bar_Bar figure  

def fig_Bar_Bar(df):
    fig = px.bar(df, 
             color='Gender', barmode='group',
             height=400)
    fig.show()    
    return fig

## DashBoard APP

In [17]:
external_stylesheets = [dbc.themes.BOOTSTRAP]

In [18]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Customer Flow'

### Page Header

In [19]:
# put the styling in place
colors = {
    'background': '#111111',
    'bodyColor':'#F2DFCE',
    'text': '#7FDBFF'
}
def get_page_heading_style():
    return {'backgroundColor': colors['background']}


def get_page_heading_title():
    return html.H1(children='Customer_Flow Dashbord',
                                        style={
                                        'textAlign': 'center',
                                        'color': colors['text']
                                    })

def get_page_heading_subtitle():
    return html.Div(children='Visualize your customer flow',
                                         style={
                                             'textAlign':'center',
                                             'color':colors['text']
                                         })

def generate_page_header():
    main_header =  dbc.Row(
                            [
                                dbc.Col(get_page_heading_title(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    subtitle_header = dbc.Row(
                            [
                                dbc.Col(get_page_heading_subtitle(),md=12)
                            ],
                            align="center",
                            style=get_page_heading_style()
                        )
    header = (main_header,subtitle_header)
    return header

### Graph Container for DASH

In [20]:
# Create a function for the graph container

def graph(i,fig):
    return dcc.Graph(id='graph{}'.format(i),figure=fig)

### Generate app layout

In [21]:
def generate_layout():
    page_header = generate_page_header()
    layout = dbc.Container(
        [
            page_header[0],
            page_header[1],
            html.Hr(),
    
            dbc.Row(
                [                
                    
                    dbc.Row(graph(1,fig_Line(group_by_date(),"Sum","Daily"))),
                    dbc.Row(graph(2,fig_Bar(group_by_Month(),"Monthly Customer flow")))
                  
                ]),
                            html.Hr(),

                 dbc.Row(
                [                
                    
                    
                    dbc.Row(graph(3,fig_Line(group_by_Year(),"Sum","Yearly"))),
                    dbc.Row(graph(5,fig_pie(group_by_Gender(),"Sum")))
                   
                  
                ]),
                                 html.Hr(),

                  dbc.Row(
                [ 
                    dbc.Row(graph(4,fig_Bar(group_by_Age(),"Age Repartition")))
                ]),
            
                 ],fluid=True,style={'backgroundColor': colors['bodyColor']}
    )
    return layout

In [22]:
app.layout = generate_layout()

## RUN the app server

In [ ]:
app.run_server(host= 'localhost',debug=False)

Dash is running on http://localhost:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Dec/2021 22:48:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 22:48:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 22:48:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 22:48:26] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2021 22:48:26] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
